In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
def bdd_connect(bdd_path):
    connection=sqlite3.connect(bdd_path)
    return connection

def bdd_req(connect,requete):
    c = connect.cursor()
    req=c.execute(requete)
    return req

def bdd_res(requete):
    for row in requete:
        print(row)

In [ ]:
df_agency=pd.read_csv("./SEM-GTFS/agency.txt")
df_stop=pd.read_csv("./SEM-GTFS/stops.txt")

In [ ]:
conn=bdd_connect("bdd_tag.db")
c=conn.cursor()

# Insertion via pandas

In [ ]:
df_agency.to_sql('agency', conn, if_exists='replace', index = False)

In [ ]:
df_stop.to_sql('stops', conn, if_exists='replace', index = False)

# Insertion des données (bulk insert)

## insertion agency (bulk insert)

In [ ]:
conn=bdd_connect("bdd_tag.db")
c=conn.cursor()
l=[]
val=values_to_insert = df_agency.iloc[0,:4]
for i in val:
    l.append(i)
sql_query = 'INSERT INTO ' + "agency" + '(agency_id,agency_name,agency_url,agency_timezone) VALUES (?,?,?,?)'
print(sql_query)
c.execute(sql_query, l)
conn.commit()
conn.close

In [ ]:
conn.close()

## Insertion Stops (bulk insert)

In [ ]:
df_stop

In [ ]:
conn=bdd_connect("bdd_tag.db")
c=conn.cursor()
l=[]
for i in range(len(df_stop)):
    l=[]
    val=values_to_insert = df_stop.iloc[i,:6]
    for j in val:
        l.append(j)
    sql_query = 'INSERT INTO ' + "stops" + '(stop_id,stop_name,stop_lat,stop_lon,zone_id,parent_station) VALUES (?,?,?,?,?,?)'
    c.execute(sql_query, l)
conn.commit()
conn.close()

## implémenter une fonction qui génère une commande insertion SQL

In [ ]:
def gen_insert_queries(table_name:str, a_dict:dict):
    insert = "INSERT INTO " + table_name + "("
    
    for i, j in enumerate(a_dict):
        if i == len(a_dict) - 1:
            insert += str(j) +") "
        else:
            insert += str(j) +", "
    
    insert += "VALUES("
    
    for i, j in enumerate(a_dict):
        if i == len(a_dict) - 1:
            insert += str(j) +")"
        else:
            insert += str(j) + ", "
            
    return insert

In [ ]:
d = {"voiture": 4, "vélo": 2}


In [ ]:
gen_insert_queries("d",d)

## implémenter une fonction qui génère des commandes d’insertion SQL

In [ ]:
def get_insert_queries(tablename:str, df: pd.DataFrame):
    
    conn=bdd_connect("bdd_tag.db")
    c=conn.cursor()
    l=[]
    list_insert=[]
    col_name=""
    
    for k in df.columns:
        if col_name=="":
            col_name=k
        else:
            col_name=col_name+','+k
            
    for i in range(len(df)):
        l=[]
        val= df_stop.iloc[i,:]
        
        for j in val:
            l.append(j)
            
        sql_query = 'INSERT INTO ' + tablename + '('+col_name+') VALUES ('+str(l)+')'
        list_insert.append(sql_query)
        
    conn.close()
    return list_insert

In [ ]:
t=get_insert_queries("stops",df_stop)
t

## implémenter une procédure qui crée un fichier SQL

In [ ]:
def gen_insert_file(filename: str, tablename, df):

    conn = sqlite3.connect(filename)
    c = conn.cursor()
    
    liste_insert = get_insert_queries(tablename, df)
    for j, l in enumerate(liste_insert):
        dico = {i: df.loc[j, i] for i in df.columns}

        c.execute(l, dico)
    conn.commit()
    conn.close()
    